# packages

In [1]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")
install("openpyxl")

# imports

In [2]:
import pandas as pd
import numpy as np
import json
import csv
import os

In [3]:
heads = ['ID', 'Title', 'model processing technique', 'source', 'source category', 'source sub-category 1', 'source sub-category 2', 'source model target category', 'target', 'target category', 'target sub-category 1', 'target sub-category 2', 'target model target category', 'Technological Readiness Level', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'use case domain','Use Case']


# Read Extraction Data from CSV

In [348]:
#df = pd.read_csv(filepath_or_buffer="../data/04 extraction consolidation results.csv",sep="|")

In [6]:
result = df['Title'].to_list()

In [7]:
result = list(dict.fromkeys(result))
print(len(result))

63


In [8]:
titles = list(df.columns)

In [11]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [12]:
metadata = metadata.drop_duplicates(subset=['Title'])

In [354]:
metadata

,Title,Publication Title,Authors,Publication Year,Document Identifier,Publisher
0,Air Quality Management: An Exemplar for Model-...,2021 ACM/IEEE International Conference on Mode...,H. S. Govindasamy; R. Jayaraman; B. Taspinar; ...,2021.0,IEEE Conferences,IEEE
1,The Research of Aero-engine Digital Twin Model...,2021 IEEE 10th Data Driven Control and Learnin...,K. Liu; P. Wang; Q. Li,2021.0,IEEE Conferences,IEEE
2,Generative and Model-driven SDK development fo...,2021 26th IEEE International Conference on Eme...,N. Braunisch; M. Ristin-Kaufmann; R. Lehmann; ...,2021.0,IEEE Conferences,IEEE
3,Models Meet Data: Challenges to Create Virtual...,2021 ACM/IEEE International Conference on Mode...,M. Van Den Brand; L. Cleophas; R. Gunasekaran;...,2021.0,IEEE Conferences,IEEE
4,Towards a Reference Architecture for Leveragin...,2020 25th IEEE International Conference on Eme...,D. Lehner; S. Wolny; A. Mazak-Huemer; M. Wimmer,2020.0,IEEE Conferences,IEEE
...,...,...,...,...,...,...
276,UAV-Assisted Three-Dimensional Spectrum Mappin...,SYMMETRY-BASEL,"Du, XF; Zhu, QM; Ding, GR; Li, J; Wu, QH; Lan,...",NaN,J,NaN
278,Decentralized Autonomous Operations and Organi...,IEEE TRANSACTIONS ON SYSTEMS MAN CYBERNETICS-S...,"Zhao, C; Dai, XY; Lv, YS; Niu, JL; Lin, YL",NaN,J,NaN
279,A virtual model knowledge updating method driv...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,"Luo, RP; Sheng, BY; Fu, GC; Huang, YZ; Jian, Y...",NaN,J,NaN
281,Data-Driven Methodology to Support Long-Lastin...,SUSTAINABILITY,"Gutierrez-Franco, E; Mejia-Argueta, C; Rabelo, L",NaN,J,NaN


In [13]:
data = df.merge(metadata, how='left', on='Title')
#unique function

In [14]:
##data = data.drop(index=0, axis = 1)
data = data.drop(data.columns[0], axis=1)

In [15]:
data

,ID,Title,model processing technique,source,source category,source sub-category 1,source sub-category 2,source model target category,target,target category,...,system lifecycle phase,twinning target,twin lifecycle phase,use case domain,Use Case,Publication Title,Authors,Publication Year,Document Identifier,Publisher
0,1,A digital twin-driven hybrid approach for the ...,Model Interpretation,"hybrid model (physical, geometric, behavior, r...",behavior + structure model,data,NaN,PT,performance degredation measurement algorithm,Software,...,Design,Individual Systems,As-Operated,Manufacturing,Transmission of CNC Machine,ROBOTICS AND COMPUTER-INTEGRATED MANUFACTURING,"Yang, X; Ran, Y; Zhang, GB; Wang, HW; Mu, ZY; ...",NaN,J,NaN
1,1,A digital twin-driven hybrid approach for the ...,Model Interpretation,runtime data,structural model,data,NaN,PT,simulation engine,Software,...,Design,Individual Systems,As-Operated,Manufacturing,Transmission of CNC Machine,ROBOTICS AND COMPUTER-INTEGRATED MANUFACTURING,"Yang, X; Ran, Y; Zhang, GB; Wang, HW; Mu, ZY; ...",NaN,J,NaN
2,2,A framework for Model-Driven Engineering of re...,Code Generation,SysML: BDD,structural model,data,NaN,PT,Knowledge Base of software digital twins (Meta...,Software,...,Design,Individual Systems,As-Designed,Transportation and Storage,limited traffic zone access control,Computing,Parri J.; Patara F.; Sampietro S.; Vicario E.,2021.0,Article,Springer
3,2,A framework for Model-Driven Engineering of re...,Model Interpretation,Fault Trees,behavior model,discrete behavior,NaN,PT,FaultTreeAnalyzer (metamodel proposed in Figur...,Software,...,Design,NaN,As-Designed,NaN,NaN,Computing,Parri J.; Patara F.; Sampietro S.; Vicario E.,2021.0,Article,Springer
4,3,A Methodology for Digital Twin Modeling and De...,Code Generation,AutomationML models,structural model,data,NaN,PT,DT runtime memory,software,...,Operate,Systems of Systems,As-Operated,NaN,NaN,Proceedings of the IEEE,G. N. Schroeder; C. Steinmetz; R. N. Rodrigues...,2021.0,IEEE Journals,IEEE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,63,Supporting the Implementation of Digital Twins...,Model to Model Transformation,BPMN Model,behavior model,discrete behavior,NaN,PT,DTDL Model,structural model,...,Implement,Processes,As-Operated,Accommodation,Business Process of Managing CO2 Levels,NaN,NaN,NaN,NaN,NaN
132,63,Supporting the Implementation of Digital Twins...,Code Generation,BPMN Model,behavior model,discrete behavior,NaN,PT,Java Templates for Device Management Microserv...,software,...,Implement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,64,Survey and Practice on Architecture and Deploy...,Model to Model Transformation,Physics-based model,behavior model,continuous behavior,NaN,PT,Data-driven model,software,...,Design,Individual Systems,As-Designed,Electricity,Oil-immersed power transformer,NaN,NaN,NaN,NaN,NaN
134,65,Towards Generating Model-Driven Speech Interfa...,Model Interpretation,Speech Interaction Model,structural model,data,NaN,PT,Speech IO Framework,software,...,Operate,Individual Systems,As-Designed,Accommodation,Smart Home use case,NaN,NaN,NaN,NaN,NaN


In [16]:
del metadata
del df
del titles

In [17]:
index = 0
print('{', end="")
for row in data:
    print(index, row)
    if index == 0:
        #print('\''+row+'\'', end="")
        index += 1
        continue
    #print(', '+'\''+row+'\'', end="")
    index += 1
print('}')

{0 ID
1 Title
2 model processing technique
3 source
4 source category
5 source sub-category 1
6 source sub-category 2
7 source model target category
8 target
9 target category
10 target sub-category 1
11 target sub-category 2
12 target model target category
13 Technological Readiness Level
14 system lifecycle phase
15 twinning target
16 twin lifecycle phase
17 use case domain
18 Use Case
19 Publication Title
20 Authors
21 Publication Year
22 Document Identifier
23 Publisher
}


# create JSON from CSV

In [25]:
arrayindexes2 = [13,15,16,17,18,19,20,21,22,23]

In [18]:
model_trafos = 'modelTrafos'
modelling_language = 'modeling language'

In [19]:
keys = ['ID', 'Title', 'model processing technique', 'source', 'source category', 'source sub-category 1', 'source sub-category 2', 'source model target category', 'target', 'target category', 'target sub-category 1', 'target sub-category 2', 'target model target category', 'Technological Readiness Level', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'use case domain', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [363]:
#keys2 = ['ID', 'Title', 'model processing technique', 'purpose of MDE application', 'source', 'source category', 'source sub-category 1', 'source sub-category 2', 'source model target category', 'target', 'target category', 'target category 2', 'target sub-category 1', 'target sub-category 2', 'target model target category', 'use case domain', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'Technological Readiness Level', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [20]:
keys2 = keys

In [21]:
out_menge_2 = {}

In [22]:
print(out_menge_2)

{}


In [26]:
for key in result:
    out_menge_2[key] = {**{modelling_language:{}, model_trafos:{}}, **{keys2[x]: None for x in arrayindexes2}}

In [24]:
out_menge_2

{}

In [27]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_2[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_2[title]['twinning target'] is None:
                    out_menge_2[title]['twinning target'] = [row['twinning target']]
                else:
                    #out_menge_2[title]['twinning target'] += ","
                    out_menge_2[title]['twinning target'].append(row['twinning target'])

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_2[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_2[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_2[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_2[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_2[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_2[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_2[title]['Publisher'] = row['Publisher']
                
            if row['twin lifecycle phase'] is not None and not pd.isna(row['twin lifecycle phase']):
                out_menge_2[title]['twin lifecycle phase'] = row['twin lifecycle phase']
            
            # throw away title and id
            arrayindexes2.append(0)
            arrayindexes2.append(1)
            rowjson = rowcopy.drop(row.iloc[arrayindexes2].index).to_json()
            try:
                out_menge_2[title][model_trafos][rownum] = rowjson
            except Exception as ex:
                out_menge_2[title][model_trafos] = {}
                out_menge_2[title][model_trafos][rownum] = rowjson
            #out_menge_2[title]['data'][rownum].append(rowjson)
            rownum+=1

In [28]:
json_object = json.dumps(out_menge_2, indent = 4) 
try:
    os.mkdir('./target/')
    os.mkdir('./target/img/')
    os.mkdir('./target/json/')
except OSError as error:
    print(error)
with open("./target/json/data_consolidated_semistructured.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: './target/'
